In [7]:
import sys
from pathlib import Path
parent_dir = Path('.').resolve().parent
sys.path.insert(0, str(parent_dir))

import numpy as np
import pandas as pd
import math
import random
import statistics
import importlib

import math_expressions as mexpr
importlib.reload(mexpr)

import methods
importlib.reload(methods)

<module 'methods' from '/Users/preetkaria/Desktop/RA/time_to_leave/methods.py'>

In [8]:
alpha_range = [2, 3, 4]
beta_range = [1, 2, 4, 6]
h_range = [1/15, 2/15, 10/15]
c_range = [20, 25, 30, 10]
N_range = range(3, 8)
n_range = [5]

In [9]:
columns = ['alpha', 'beta', 'N', 'n', 'h', 'c', 'mean_n', 'std_n', 'alpha_hat', 'beta_hat', 'u_star', 'u_star_hat', 'z', 'u', 'optimal_cost', 'actual_cost', 'intervals_str']
df = pd.DataFrame(columns=columns)

In [10]:
for _ in range(200):
    alpha = random.choice(alpha_range)
    beta = random.choice(beta_range)
    h = random.choice(h_range)
    c = random.choice(c_range)
    N = random.choice(N_range)
    n = random.choice(n_range)
    intervals = np.random.gamma(shape=alpha, scale=beta, size=N+n)

    mean_n = statistics.mean(intervals[:n])
    std_n = statistics.stdev(intervals[:n])

    alpha_hat, beta_hat = methods.gamma_estimate_parameters(n, intervals)
    alpha_hat = round(alpha_hat) # fractional alpha has no solution for u*, hencing rounding it.

    u = methods.cal_actual_time(n, intervals)
    u_star = methods.get_u_star(N, alpha, beta, h, c)
    u_star_hat = methods.get_u_star(N, alpha_hat, beta_hat, h, c)
    z=u_star / u_star_hat

    optimal_cost = methods.cal_cost(c, h, u, u_star)
    actual_cost = methods.cal_cost(c, h, u, u_star_hat)

    intervals_str = '_'.join([str(x) for x in intervals])
    dic = {'alpha': alpha,
           'beta': beta,
           'N': N,
           'n':n,
           'h': h,
           'c': c,
           'mean_n': mean_n,
           'std_n': std_n,
           'alpha_hat': alpha_hat,
           'beta_hat': beta_hat,
           'u_star': u_star,
           'u_star_hat': u_star_hat,
           'z': z,
           'u': u,
           'optimal_cost': optimal_cost,
           'actual_cost': actual_cost,
           'intervals_str': intervals_str}
    print(dic)
    df.loc[len(df)] = dic


{'alpha': 4, 'beta': 6, 'N': 5, 'n': 5, 'h': 0.13333333333333333, 'c': 25, 'mean_n': 19.214556907952534, 'std_n': 8.736612195826527, 'alpha_hat': 5, 'beta_hat': 3.9724253349123018, 'u_star': 79.50987743759983, 'u_star_hat': 66.56318168465629, 'z': 1.194502357388483, 'u': 120.63658902149172, 'optimal_cost': 5.483561544518919, 'actual_cost': 7.2097876449113905, 'intervals_str': '28.173785338102938_28.359584955468065_17.335397098373445_12.870643082969663_9.333374064848568_20.761472232207197_44.54019409887413_15.371836182150208_28.95762297393832_11.005463534321883'}
{'alpha': 3, 'beta': 4, 'N': 5, 'n': 5, 'h': 0.6666666666666666, 'c': 25, 'mean_n': 12.992619044466712, 'std_n': 5.967654310603978, 'alpha_hat': 5, 'beta_hat': 2.741009941797457, 'u_star': 46.71946149543203, 'u_star_hat': 55.88911439667533, 'z': 0.835931325800561, 'u': 86.87629236928409, 'optimal_cost': 26.771220582568038, 'actual_cost': 20.658118648405836, 'intervals_str': '20.390684440092205_7.781923056848202_12.0834095808070

In [11]:
display(df.head())
methods.plot_plotly(data=df['actual_cost'], data_label='Cost')

,alpha,beta,N,n,h,c,mean_n,std_n,alpha_hat,beta_hat,u_star,u_star_hat,z,u,optimal_cost,actual_cost,intervals_str
0,4,6,5,5,0.133333,25,19.214557,8.736612,5,3.972425,79.509877,66.563182,1.194502,120.636589,5.483562,7.209788,28.173785338102938_28.359584955468065_17.33539...
1,3,4,5,5,0.666667,25,12.992619,5.967654,5,2.741010,46.719461,55.889114,0.835931,86.876292,26.771221,20.658119,20.390684440092205_7.781923056848202_12.083409...
2,3,4,4,5,0.066667,20,12.609502,7.525812,3,4.491680,21.694680,24.871232,0.872280,94.679091,4.865627,4.653857,15.40608457326099_6.8971592235863355_15.448465...
3,4,6,6,5,0.066667,30,19.175787,6.062907,10,1.916941,89.829010,83.239638,1.079161,141.873474,3.469631,3.908922,21.114228846495497_21.204394544100456_22.99219...
4,3,1,6,5,0.066667,10,3.557652,2.531820,2,1.801782,9.155041,9.594768,0.954170,19.701268,0.703082,0.673767,3.2132121131017914_3.800249506909465_0.9399760...


In [12]:
df.to_csv('../data/gen_data.csv', index=False)